###### References:  
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 9: A Pythonic Object
    
# Object Representations

`repr()` Return a string representing the object as the developer wants to see it.

`str()` Return a string reprenting the object as the user wants to see it.

## Verctor Class Redux


In [1]:
from array import array
import math

In [2]:
class Vector2d:
    typecode = 'd'  # used when converting instances to/from bytes

    def __init__(self, x, y):
        self.x = float(x)    # converting to float to avoid/catch errors early
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # makes the object iterable, and make unpacking works; by using genexp

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # interpolates the components with {!r}

    def __str__(self):
        return str(tuple(self))  # build a tuple for display

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))  # convert from an array

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # this works, but also returns true with other iterables with same numeric values

    def __abs__(self):
        return math.hypot(self.x, self.y)  # The magnitude is the length of the hypotenuse of the triangle formed

    def __bool__(self):
        return bool(abs(self))  # uses the magnitude and converts it to bool.

In [3]:
v1 = Vector2d(3, 4)

# the components can be accessed directly as attributes
print(v1.x, v1.y)

3.0 4.0


In [4]:
x, y = v1
x, y

(3.0, 4.0)

In [5]:
v1

Vector2d(3.0, 4.0)

In [6]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [7]:
print(v1)

(3.0, 4.0)


In [8]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [9]:
abs(v1)

5.0

In [10]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

# An Alternative Constructor

Because we export a `Vector2d` as bytes, we need a method that imports a `Vector2d` from a binary sequence.

In [11]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))


    @classmethod  
    def frombytes(cls, octets):  # no `self`, the class itself is passed as `cls`
        typecode = chr(octets[0])  # read typecode from the first byte
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)  # unpack, invoke `cls` to build a new instance

In [12]:
v1_clone = Vector2d.frombytes(bytes(v1))
v1_clone

Vector2d(3.0, 4.0)

In [13]:
v1 == v1_clone

True

# `classmethod` Versus `staticmethod`
`classmethod` define a method that operates on the class and not on instances

`staticmethod` changes a method so that it receives no special first argument.

In [14]:
class Demo:
    @classmethod  
    def klassmethod(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [15]:
Demo.klassmethod()

(__main__.Demo,)

In [16]:
Demo.klassmethod('spam')

(__main__.Demo, 'spam')

In [17]:
Demo.statmeth()

()

In [18]:
Demo.statmeth('spam')

('spam',)

# Formatted Displays
the format specifier

In [19]:
# MYR to SGD currency conversino rate
myr = 1/3.2
myr

0.3125

In [20]:
format(myr, '0.4f')

'0.3125'

In [21]:
'1 MYR = {rate:0.2f} SGD'.format(rate=myr)

'1 MYR = 0.31 SGD'

In [22]:
format(42, 'b')

'101010'

In [23]:
format(42, 'x')

'2a'

In [24]:
format(42, 'X')

'2A'

In [25]:
format(2/3, '.1%')

'66.7%'

In [26]:
from datetime import datetime

In [27]:
now = datetime.now()
format(now, '%H:%M:%S')

'14:23:29'

In [28]:
"It's now {:%I:%M %p}".format(now)

"It's now 02:23 PM"

`__format__` inherits `str(object)` if not specifited:

In [29]:
format(v1)

'(3.0, 4.0)'

In [30]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [31]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        """for use to generate polar coordinates"""
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            """use polar coordinates"""
            fmt_spec = fmt_spec[:-1] # remove the 'p' suffix
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>' 
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords) # generate iterable with components as formatted strings
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [32]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [33]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [34]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

# A Hashable Vector2d

In [35]:
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [36]:
set([v1])

TypeError: unhashable type: 'Vector2d'

Making the components read-only:

In [37]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x) # make an attribute private using two underscores
        self.__y = float(y)

    @property     # marks the getter method of a property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y) # ^ operator to mix the hashes of the components

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [38]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [39]:
len(set([v1, v2]))

2

In [40]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

Implement with a sensible scaler numeric value:

In [41]:
print(v1.x, v1.y)

3.0 4.0


## Private and "Protected" Attributes in Python
In Python, we use two leading underscores to signify a `private` attribute.

### Name mangling:
Python sotres the name in the instance `__dict__` prefixed with a leading underscore and the classname, i.e. _Classname__privateattribute.

In [42]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

Attributes with a single _ prefix are called "protected" by some.

## Saving Space with the __slots__ Class Attribute
By default, Python stores instance attributes in a per-instance `dict`; which have a significant memory over head becuase of the underlying hash table used to provide fast access.

The `__slots__` class attribute let the interpreter store the instance attributes in a tuple instead of a dict.

#### Warning: `__slots__` attribute method is not inheritable
Other issues:
- Instances will only be able to have attributes listed in `__slots__`
- Instances cannot be targets of weak references unless you include `__weakref__` in `__slots__`.

    class Vector2d:
        __slots__ = ('__x', '__y')

        typecode = 'd'



    time python3 mem_test.py vector2d_v3.py

# Overriding Class Attributes
Customizing an instance by setting the typecode attribute that was formerly inherited from the class:

In [43]:
dumpd = bytes(v1)
dumpd

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [44]:
len(dumpd)

17

In [45]:
v1.typecode = 'f'

dumpf = (bytes(v1))
dumpf

b'f\x00\x00@@\x00\x00\x80@'

In [46]:
len(dumpf)

9

In [47]:
Vector2d.typecode

'd'

Overriding the default typecode:

In [48]:
class ShortVector(Vector2d):
    typecode = 'f'

In [49]:
sv = ShortVector(1/11, 1/27)
sv

ShortVector(0.09090909090909091, 0.037037037037037035)

In [50]:
len(bytes(sv))

9